In [291]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
import itertools
import seaborn as sn
from textblob import TextBlob
import glob

# warning
import warnings
warnings.filterwarnings("ignore")

In [292]:
df = pd.read_csv('../data/final_data.csv')
df.head(3)

,date,full_text,geo,truncated,display_text_range,user,tweet_id,entities,possibly_sensitive,longitude,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
0,2020-03-30,Definitely making a routine to stop from going...,"[41.42362498, -122.39264801]",False,"[0, 239]","{'id': 988896150571790336, 'id_str': '98889615...",1244710975422574592,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-122.392648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0625
1,2020-03-30,This is my 'self-isolating' face. #quarantine ...,"[34.1722, -118.378]",False,"[0, 140]","{'id': 1008423932267921408, 'id_str': '1008423...",1244710063589089281,"{'hashtags': [{'text': 'quarantine', 'indices'...",False,-118.378000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,2020-03-30,Corona virus testing and treatment is availabl...,"[33.97977, -118.18884]",False,"[0, 233]","{'id': 107183997, 'id_str': '107183997', 'name...",1244708953994469377,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,-118.188840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4000


In [293]:
irrelevant_cols = ['full_text', 'geo', 'display_text_range', 'user', 'tweet_id',
       'entities', 'longitude', 'latitude', 'state', 'preprocess_full_text']

In [294]:
relevant_cols = df.drop(columns=irrelevant_cols)
relevant_cols.head(2)

,date,truncated,possibly_sensitive,county,retail_rec,groc_pharm,parks,transit,workplaces,residential,...,yoursel,youth,youtube,yrxltdq,yummy,yumyum,yup,zero,zoom,sentiment
0,2020-03-30,False,False,Siskiyou County,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0625
1,2020-03-30,False,False,Los Angeles County,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000


In [295]:
# Length before dropping missing values
len(relevant_cols)

17531

In [296]:
# Drop rows with NaN values
no_na = relevant_cols.dropna()

In [297]:
# Length after dropping missing values
len(no_na)

15923

In [298]:
# One hot encode categorical columns
dummy_cols = ['county', 'truncated', 'possibly_sensitive']
ohe_df = pd.get_dummies(no_na, drop_first=True, columns=dummy_cols)
ohe_df.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Shasta County,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True
0,2020-03-30,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25,-0.0625,0.0,...,0,1,0,0,0,0,0,0,0,0
1,2020-03-30,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00,0.0000,0.0,...,0,0,0,0,0,0,0,0,0,0


In [299]:
# Add county back
ohe_df['county'] = no_na['county']
ohe_df.head(1)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,county
0,2020-03-30,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25,-0.0625,0.0,...,1,0,0,0,0,0,0,0,0,Siskiyou County


In [300]:
y_labels = ohe_df.loc[:, ['retail_rec',
       'groc_pharm', 'parks', 'transit', 'workplaces', 'residential', 'public']]
y_labels.head()

,retail_rec,groc_pharm,parks,transit,workplaces,residential,public
0,-33.0,-12.0,-2.0,-12.0,-40.0,10.0,-24.25
1,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
2,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
4,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00
5,-42.0,-23.0,-34.0,-52.0,-51.0,22.0,-42.00


In [301]:
# minus_first_20 = df.iloc[:, 20:]
# minus_first_20['date'] = df['date']
# minus_first_20['county'] = df['county']
# new_df = minus_first_20
# new_df.head()

In [302]:
grouped = ohe_df.groupby(['county','date']).mean()
grouped.head(2)

retail_rec  groc_pharm  parks  transit  workplaces  \
county         date                                                             
Alameda County 2020-03-30       -48.0       -23.0   -1.0    -71.0       -64.0   
               2020-04-01       -48.0       -18.0    0.0    -70.0       -65.0   

                           residential  public  polarity  aad  abc  ...  \
county         date                                                 ...   
Alameda County 2020-03-30         27.0  -51.50  0.233333  0.0  0.0  ...   
               2020-04-01         28.0  -50.25  0.221645  0.0  0.0  ...   

                           county_Shasta County  county_Siskiyou County  \
county         date                                                       
Alameda County 2020-03-30                   0.0                     0.0   
               2020-04-01                   0.0                     0.0   

                           county_Solano County  county_Sonoma County  \
county         date                                                     
Alameda County 2020-03-30                   0.0                   0.0   
               2020-04-01                   0.0                   0.0   

                           county_Stanislaus County  county_Tulare County  \
county         date                                                         
Alameda County 2020-03-30                       0.0                   0.0   
               2020-04-01                       0.0                   0.0   

                           county_Ventura County  county_Yolo County  \
county         date                                                    
Alameda County 2020-03-30                    0.0                 0.0   
               2020-04-01                    0.0                 0.0   

                           truncated_True  possibly_sensitive_True  
county         date                                                 
Alameda County 2020-03-30             0.0                      0.0  
               2020-04-01             0.0                      1.0  

[2 rows x 2332 columns]

In [303]:
len(grouped)

2540

In [304]:
reset_ind = grouped.reset_index()
reset_ind.drop(columns=['county'], inplace=True)

In [305]:
final_df = reset_ind
final_df.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Shasta County,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True
0,2020-03-30,-48.0,-23.0,-1.0,-71.0,-64.0,27.0,-51.50,0.233333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-04-01,-48.0,-18.0,0.0,-70.0,-65.0,28.0,-50.25,0.221645,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [306]:
mobility_columns = list(final_df.columns)[1:8]
mobility_columns

['retail_rec',
 'groc_pharm',
 'parks',
 'transit',
 'workplaces',
 'residential',
 'public']

In [307]:
y_public = final_df['public']
y_residential = final_df['residential']

In [308]:
# Convert date column to pd.DateTime object
final_df['date'] = pd.to_datetime(final_df['date'])
type(final_df['date'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [309]:
final_df['date_num'] = (final_df['date'] - pd.to_datetime('03/30/2020'))/np.timedelta64(1, 'D')
final_df = final_df.sort_values(by='date')
final_df.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,date_num
0,2020-03-30,-48.0,-23.0,-1.0,-71.0,-64.0,27.0,-51.50,0.233333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1028,2020-03-30,-34.0,-18.0,-30.0,-35.0,-44.0,17.0,-32.75,0.068750,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


In [310]:
before_mid_june = final_df[final_df['date'] < pd.to_datetime('2020-06-15')]
mid_june_onwards = final_df[final_df['date'] >= pd.to_datetime('2020-06-15')]
mid_june_onwards.tail(5)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,date_num
971,2020-09-20,-27.0,-10.0,14.0,-38.0,-17.0,6.0,-23.0,0.146667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,174.0
2513,2020-09-20,-22.0,-14.0,12.0,-11.0,-17.0,5.0,-16.0,0.321528,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,174.0
1942,2020-09-20,-17.0,-2.0,13.0,-33.0,-12.0,3.0,-16.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,174.0
1202,2020-09-20,-22.0,-9.0,-31.0,-30.0,-23.0,5.0,-21.0,0.125805,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.176471,174.0
277,2020-09-20,-30.0,-12.0,12.0,-38.0,-16.0,6.0,-24.0,0.016667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,174.0


In [311]:
less_cols = list(before_mid_june.columns)[-39:]

In [312]:
few_cols_train = before_mid_june.loc[:, less_cols]
few_cols_test = mid_june_onwards.loc[:, less_cols]
y_public_train = np.array(y_public_train)
y_public_test = np.array(y_public_test)

In [313]:
few_cols_train.columns

Index(['sentiment', 'county_Contra Costa County', 'county_Fresno County',
       'county_Humboldt County', 'county_Imperial County',
       'county_Kern County', 'county_Kings County', 'county_Lake County',
       'county_Los Angeles County', 'county_Madera County',
       'county_Marin County', 'county_Mendocino County',
       'county_Merced County', 'county_Monterey County', 'county_Napa County',
       'county_Nevada County', 'county_Orange County', 'county_Placer County',
       'county_Riverside County', 'county_Sacramento County',
       'county_San Bernardino County', 'county_San Diego County',
       'county_San Joaquin County', 'county_San Luis Obispo County',
       'county_San Mateo County', 'county_Santa Barbara County',
       'county_Santa Clara County', 'county_Santa Cruz County',
       'county_Shasta County', 'county_Siskiyou County',
       'county_Solano County', 'county_Sonoma County',
       'county_Stanislaus County', 'county_Tulare County',
       'county_Ventur

In [314]:
few_cols_train.head(2)

,sentiment,county_Contra Costa County,county_Fresno County,county_Humboldt County,county_Imperial County,county_Kern County,county_Kings County,county_Lake County,county_Los Angeles County,county_Madera County,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,date_num
0,0.233333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1028,0.068750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


In [315]:
final_df.head(2)

,date,retail_rec,groc_pharm,parks,transit,workplaces,residential,public,polarity,aad,...,county_Siskiyou County,county_Solano County,county_Sonoma County,county_Stanislaus County,county_Tulare County,county_Ventura County,county_Yolo County,truncated_True,possibly_sensitive_True,date_num
0,2020-03-30,-48.0,-23.0,-1.0,-71.0,-64.0,27.0,-51.50,0.233333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1028,2020-03-30,-34.0,-18.0,-30.0,-35.0,-44.0,17.0,-32.75,0.068750,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0


### Try building a linear model that's not aggregated by county, aggregate by state instead

In [316]:
no_county = final_df.loc[:, ['date', 'public', 'sentiment', 'truncated_True', 'possibly_sensitive_True', 'date_num']]
no_county = no_county.groupby(['date'], as_index=False).mean()
no_county.head(3)

,date,public,sentiment,truncated_True,possibly_sensitive_True,date_num
0,2020-03-30,-40.000000,0.145458,0.0,0.044643,0.0
1,2020-03-31,-38.041667,0.179263,0.0,0.113997,1.0
2,2020-04-01,-41.325000,0.086206,0.0,0.233710,2.0


In [317]:
X_tr, X_te = get_X_train_test(no_county)
X_tr, X_te, y_public_tr, y_public_te = get_X_y_train_test(X_tr, X_te, cols_to_drop=['date', 'public'])
y_public_tr[:2]

array([-40.        , -38.04166667])

In [318]:
lm_agg_state = linear_model.LinearRegression()
lm.fit(X_tr, y_public_tr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [319]:
y_public_pred_state = lm.predict(X_te)

In [320]:
get_accuracy(y_public_pred_state, y_public_te, margin=5)

0.14285714285714285

### Split into train & test

In [321]:
def get_X_train_test(df):
    X_train = df[df['date'] < pd.to_datetime('2020-06-15')]
    X_test = df[df['date'] >= pd.to_datetime('2020-06-15')]
    return (X_train, X_test)

In [322]:
def get_X_y_train_test(df_train, df_test, cols_to_drop = mobility_columns+['date'], public=True):
    X_train = df_train.drop(columns=cols_to_drop)
    y_public_train = np.array(df_train['public'])
#     y_residential_train = df_train['residential']
    X_test = df_test.drop(columns=cols_to_drop)
    y_public_test = np.array(df_test['public'])
#     y_residential_test = df_test['residential']
    return (X_train, X_test, y_public_train, y_public_test)

In [323]:
plt.scatter(before_mid)

NameError: name 'before_mid' is not defined

In [ ]:
plt.scatter(final_df['truncated_True'], final_df['public'])

### Build the linear model

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(few_cols_train, y_public_train)

In [ ]:
y_public_pred = lm.predict(few_cols_test)

In [ ]:
sum(y_public_pred == y_public_test) / len(y_public_test)

In [ ]:
y_public_pred[:20]

In [ ]:
np.array(y_public_test[:20])

In [ ]:
def get_accuracy(pred_y, actual_y, margin=0):
    accuracies = []
    for i in range(len(pred_y)):
        if (abs(pred_y[i] - actual_y[i]) <= margin):
            accuracies.append(1)
        else:
            accuracies.append(0)
    return sum(accuracies)/len(accuracies)

In [ ]:
residuals = y_public_test - y_public_pred
plt.scatter(y_public_pred, residuals)

In [ ]:
get_accuracy(y_public_pred, y_public_test, margin=5)